<a href="https://colab.research.google.com/github/zq88577727/moxingweitiao/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


In [6]:
pip install wandb

In [7]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sunjieqiong7 (sunjieqiong7-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
import random

import wandb

# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="sunjieqiong7-",
    # Set the wandb project where this run will be logged.
    project="111",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
)

# Simulate training.
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset

    # Log metrics to wandb.
    run.log({"acc": acc, "loss": loss})

# Finish the run and upload any remaining data.
run.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sunjieqiong7 (sunjieqiong7-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


acc,▁▄▂▃▇▇█▇
loss,█▄▅▄▃▂▁▂
acc,0.84723
loss,0.16295


### Installation

In [9]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [10]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Load up `Llama 3.1 8B Instruct`, and set parameters

In [17]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import os

# 设置环境变量以避免内存碎片
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 设置更保守的参数
max_seq_length = 512
lora_rank = 16

print("正在加载模型...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = False,
    gpu_memory_utilization = 0.4,
)

# 设置LoRA参数
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

print("模型加载完成")

正在加载模型...
==((====))==  Unsloth 2025.3.2: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.3.2 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


模型加载完成


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [21]:
import re
from datasets import load_dataset, Dataset

# 系统提示设置为医学AI助手
SYSTEM_PROMPT = """你是一个专业的医学AI助手。请基于用户的问题，先进行详细的医学推理，然后给出最终答案。
你的回答应该包含两部分：
1. 推理部分：详细分析问题，考虑相关的医学知识、可能的诊断、治疗方案等。
2. 答案部分：基于推理给出简明扼要的最终答案。

请使用以下格式回答：
<reasoning>
详细的医学推理过程
</reasoning>
<answer>
最终答案
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

# 提取XML标签中的内容
def extract_xml_reasoning(text: str) -> str:
    try:
        reasoning = re.search(r"<reasoning>(.*?)</reasoning>", text, re.DOTALL).group(1)
        return reasoning.strip()
    except:
        return ""

def extract_xml_answer(text: str) -> str:
    try:
        answer = re.search(r"<answer>(.*?)</answer>", text, re.DOTALL).group(1)
        return answer.strip()
    except:
        return ""

# 加载医学数据集
def load_medical_dataset(lang="zh") -> Dataset:
    print(f"正在加载医学数据集 ({lang})...")
    data = load_dataset('FreedomIntelligence/medical-o1-reasoning-SFT', lang)['train']

    # 将数据转换为适合训练的格式
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['Question']}  # 使用正确的字段名 'Question'
        ],
        'completion': XML_COT_FORMAT.format(
            reasoning=x['Complex_CoT'],  # 使用 'Complex_CoT' 作为推理部分
            answer=x['Response']         # 使用 'Response' 作为答案部分
        ),
        'reasoning': x['Complex_CoT'],
        'answer': x['Response']
    })

    print(f"数据集大小: {len(data)}条")
    return data

# 加载数据集 - 使用中文数据集
dataset = load_medical_dataset("zh")

# 奖励函数

# 1. 医学正确性评估函数
def medical_correctness_reward_func(completions, **kwargs) -> list[float]:
    """评估医学回答的正确性（简化版本）"""
    responses = [completion[0]['content'] for completion in completions]
    scores = []

    for response in responses:
        # 提取推理和答案部分
        reasoning = extract_xml_reasoning(response)
        answer = extract_xml_answer(response)

        if reasoning and answer:
            # 计算推理和答案的长度
            reasoning_length = len(reasoning.split())
            answer_length = len(answer.split())

            # 推理应该比答案长
            if reasoning_length > answer_length * 2 and reasoning_length > 100:
                scores.append(1.0)
            elif reasoning_length > answer_length and reasoning_length > 50:
                scores.append(0.8)
            elif reasoning_length > 30:
                scores.append(0.5)
            else:
                scores.append(0.2)
        else:
            scores.append(0.0)

    return scores

# 2. 医学术语评估函数
def medical_terminology_reward_func(completions, **kwargs) -> list[float]:
    """评估医学术语的使用情况"""
    # 定义一些常见的医学术语和关键词
    medical_terms = [
        "诊断", "症状", "治疗", "病因", "病理", "生理", "解剖",
        "药物", "剂量", "副作用", "禁忌症", "适应症", "并发症", "检查",
        "实验室", "影像", "手术", "康复", "预防", "流行病学", "病史",
        "体征", "鉴别诊断", "病程", "疗效", "随访", "复发", "缓解",
        "病毒", "细菌", "感染", "免疫", "炎症", "代谢", "内分泌", "神经",
        "心血管", "呼吸", "消化", "泌尿", "血液", "肿瘤", "基因", "蛋白质"
    ]

    responses = [completion[0]['content'] for completion in completions]
    scores = []

    for response in responses:
        term_count = sum(response.count(term) for term in medical_terms)

        # 根据术语数量给予奖励，但设置上限
        score = min(1.0, term_count / 25.0)  # 最多25个术语得满分
        scores.append(score)

    return scores

# 3. 严格格式检查函数
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """检查输出是否严格符合XML格式要求"""
    pattern = r"<reasoning>\s*.*?\s*</reasoning>\s*<answer>\s*.*?\s*</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [bool(re.search(pattern, r, re.DOTALL)) for r in responses]
    return [1.0 if match else 0.0 for match in matches]

# 4. 软格式检查函数
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """检查输出是否包含XML标签，但格式要求较宽松"""
    responses = [completion[0]["content"] for completion in completions]
    scores = []

    for response in responses:
        has_reasoning_tag = "<reasoning>" in response and "</reasoning>" in response
        has_answer_tag = "<answer>" in response and "</answer>" in response

        if has_reasoning_tag and has_answer_tag:
            scores.append(1.0)
        elif has_reasoning_tag or has_answer_tag:
            scores.append(0.5)
        else:
            scores.append(0.0)

    return scores

# 5. XML标签计数函数
def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    """计算XML标签的使用情况"""
    responses = [completion[0]["content"] for completion in completions]
    scores = []

    for response in responses:
        score = 0.0
        if "<reasoning>" in response:
            score += 0.25
        if "</reasoning>" in response:
            score += 0.25
        if "<answer>" in response:
            score += 0.25
        if "</answer>" in response:
            score += 0.25
        scores.append(score)

    return scores

# 6. 推理深度评估函数
def reasoning_depth_reward_func(completions, **kwargs) -> list[float]:
    """评估医学推理的深度和质量"""
    depth_indicators = [
        "首先", "其次", "然后", "接着", "此外", "最后", "总结",
        "因为", "所以", "但是", "然而", "尽管", "虽然", "如果",
        "可能", "可能性", "考虑", "分析", "评估", "比较", "对比",
        "原因", "机制", "病理", "生理", "解释", "证据", "研究表明"
    ]

    responses = [completion[0]['content'] for completion in completions]
    scores = []

    for response in responses:
        # 提取推理部分
        reasoning = extract_xml_reasoning(response)

        if reasoning:
            # 计算深度指标的出现次数
            depth_count = sum(reasoning.count(indicator) for indicator in depth_indicators)

            # 检查推理的结构性
            has_structure = any(indicator in reasoning for indicator in ["首先", "其次", "然后", "最后"])

            # 计算推理的段落数（通过换行符估计）
            paragraphs = reasoning.count("\n") + 1

            # 综合评分
            depth_score = min(1.0, depth_count / 15.0)  # 深度指标得分
            structure_score = 0.5 if has_structure else 0.0  # 结构得分
            paragraph_score = min(0.5, paragraphs / 5.0)  # 段落得分

            total_score = (depth_score + structure_score + paragraph_score) / 2.0  # 平均得分，最高1.0
            scores.append(total_score)
        else:
            scores.append(0.0)

    return scores

# 打印数据集示例
print("\n数据集示例:")
example = dataset[0]
print(f"问题: {example['prompt'][1]['content']}")
print(f"推理: {example['reasoning']}")
print(f"答案: {example['answer']}")
print(f"完整回答: {example['completion']}")

正在加载医学数据集 (zh)...


Map:   0%|          | 0/24772 [00:00<?, ? examples/s]

数据集大小: 24772条

数据集示例:
问题: 根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？
推理: 这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
答案: 从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [39]:
from unsloth import FastLanguageModel
import torch
from datasets import Dataset, load_dataset
import wandb
import time
import re
import numpy as np
import os
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import json
import inspect
import linecache

# 修复 inspect.findsource 函数，避免 OSError: could not get source code 错误
original_findsource = inspect.findsource

def patched_findsource(object):
    try:
        return original_findsource(object)
    except OSError:
        # 如果无法获取源代码，返回空列表和0
        return [], 0

# 应用补丁
inspect.findsource = patched_findsource

# 设置环境变量 - 必须在导入任何其他库之前设置
os.environ["UNSLOTH_DISABLE_TRITON"] = "1"  # 禁用Triton优化
os.environ["UNSLOTH_DISABLE_GRADIENT_CHECKPOINTING"] = "1"  # 禁用梯度检查点
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # 强制CUDA操作同步执行
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 禁用tokenizer并行
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"  # 禁用警告
os.environ["TORCH_USE_CUDA_DSA"] = "0"  # 禁用CUDA动态共享内存分配
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"  # 限制CUDA内存分配大小
os.environ["PYTORCH_NO_CUDA_MEMORY_CACHING"] = "1"  # 禁用CUDA内存缓存

# 尝试禁用triton
try:
    import triton
    print("已导入triton，版本:", triton.__version__ if hasattr(triton, "__version__") else "未知")
except:
    print("triton未安装或无法导入")

# 设置PyTorch内存管理
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
if hasattr(torch.backends, "cuda"):
    if hasattr(torch.backends.cuda, "matmul"):
        torch.backends.cuda.matmul.allow_tf32 = False
    if hasattr(torch.backends.cuda, "allow_tf32"):
        torch.backends.cuda.allow_tf32 = False

# 诊断信息
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA是否可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA版本: {torch.version.cuda}")
    print(f"GPU型号: {torch.cuda.get_device_name(0)}")
    print(f"可用GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# 初始化wandb
wandb.init(project="medical-model-grpo", name="medical-sft-test-small-dataset")

# 加载医学数据集
print("正在加载数据集...")
try:
    dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en")
    print(f"数据集大小: {len(dataset['train'])}条")
except Exception as e:
    print(f"加载数据集时出错: {e}")
    print("尝试使用缓存加载...")
    try:
        dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", use_auth_token=False)
        print(f"使用缓存加载成功，数据集大小: {len(dataset['train'])}条")
    except Exception as e2:
        print(f"使用缓存加载失败: {e2}")
        raise

# 检查数据集结构
print("\n===== 数据集结构检查 =====")
print(f"数据集特征: {dataset['train'].features}")
print(f"数据集第一个样本: {dataset['train'][0]}")
print(f"数据集第一个样本类型: {type(dataset['train'][0])}")
if isinstance(dataset['train'][0], dict):
    print("字典键: ", list(dataset['train'][0].keys()))
    for key in dataset['train'][0].keys():
        print(f"键 '{key}' 的值: {dataset['train'][0][key]}")
        print(f"键 '{key}' 的类型: {type(dataset['train'][0][key])}")
elif isinstance(dataset['train'][0], str):
    print("数据集样本是字符串类型")
    try:
        # 尝试解析为JSON
        json_data = json.loads(dataset['train'][0])
        print("JSON解析结果:", json_data)
    except:
        print("不是有效的JSON格式")
        print("样本内容前200个字符:", dataset['train'][0][:200])
print("===== 数据集结构检查结束 =====\n")

# 定义系统提示
system_prompt = """你是一个专业的医学AI助手。请根据用户的医学问题，先提供详细的推理过程，然后给出明确的答案。
你的回答必须遵循以下格式：
<reasoning>详细的医学推理过程</reasoning>
<answer>最终的医学答案</answer>
"""

# 使用少量数据进行测试
test_size = 500  # 只使用500条数据进行测试
full_dataset = dataset["train"].select(range(test_size))
print(f"使用测试数据集: {len(full_dataset)}条")

# 加载模型
print("正在加载模型...")
try:
    # 尝试使用最简单的方式加载模型
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/mistral-7b-bnb-4bit",
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )
    print("模型加载完成")
except Exception as e:
    print(f"加载模型时出错: {e}")
    print("尝试使用备用方法加载...")
    try:
        # 尝试使用更多参数
        from transformers import AutoTokenizer, BitsAndBytesConfig

        # 配置量化参数
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )

        # 尝试使用transformers直接加载
        print("尝试使用transformers直接加载模型...")
        from transformers import AutoModelForCausalLM

        tokenizer = AutoTokenizer.from_pretrained("unsloth/mistral-7b-bnb-4bit")
        model = AutoModelForCausalLM.from_pretrained(
            "unsloth/mistral-7b-bnb-4bit",
            quantization_config=bnb_config,
            device_map="auto"
        )

        # 尝试将模型转换为Unsloth格式
        try:
            print("尝试将模型转换为Unsloth格式...")
            model = FastLanguageModel.get_peft_model(
                model,
                r=16,
                target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                               "gate_proj", "up_proj", "down_proj"],
                lora_alpha=16,
                lora_dropout=0,
                bias="none",
                use_gradient_checkpointing=False,
            )
            print("模型转换成功")
        except Exception as e3:
            print(f"模型转换失败: {e3}")
            print("将继续使用原始transformers模型")

        print("使用备用方法加载模型成功")
    except Exception as e2:
        print(f"使用备用方法加载模型失败: {e2}")
        raise

# 准备训练数据
print("正在准备训练数据集...")
def prepare_training_data(data):
    processed_data = []
    batch_size = 100  # 减小批处理大小，避免内存问题

    # 直接处理数据集中的每个样本
    for i in range(0, len(data)):
        if i % 100 == 0:
            print(f"处理数据 {i} 到 {min(i+100, len(data))}...")

        try:
            # 获取样本
            sample = data[i]

            # 打印第一个样本的详细信息
            if i == 0:
                print(f"样本类型: {type(sample)}")
                if isinstance(sample, dict):
                    print(f"样本键: {list(sample.keys())}")

            # 处理字典类型的样本
            if isinstance(sample, dict):
                if "Question" in sample and "Complex_CoT" in sample and "Response" in sample:
                    question = sample["Question"]
                    complex_cot = sample["Complex_CoT"]
                    response = sample["Response"]

                    # 构建训练样本
                    prompt = f"{system_prompt}\n\n用户: {question}\n\n助手: "
                    completion = f"<reasoning>{complex_cot}</reasoning>\n<answer>{response}</answer>"

                    processed_data.append({
                        "text": prompt + completion,
                        "prompt": prompt,
                        "completion": completion
                    })

                    # 打印第一个处理成功的样本
                    if len(processed_data) == 1:
                        print("\n===== 第一个处理成功的样本 =====")
                        print(f"提示: {prompt[:100]}...")
                        print(f"回复: {completion[:100]}...")
                        print("===== 第一个处理成功的样本结束 =====\n")
                else:
                    print(f"样本缺少必要字段: {list(sample.keys()) if isinstance(sample, dict) else '非字典类型'}")
            else:
                print(f"未知样本类型: {type(sample)}")
        except Exception as e:
            print(f"处理样本 {i} 时出错: {e}")
            continue

    print(f"准备完成，共{len(processed_data)}条数据")
    if len(processed_data) == 0:
        print("警告: 处理后的数据为空！请检查数据集格式和处理逻辑。")

    return processed_data

# 处理数据
try:
    processed_data = prepare_training_data(full_dataset)
    if len(processed_data) == 0:
        raise ValueError("处理后的数据为空，无法继续训练")

    train_dataset = Dataset.from_list(processed_data)
    print(f"创建训练数据集成功，大小: {len(train_dataset)}条")
except Exception as e:
    print(f"处理数据时出错: {e}")
    raise

# 添加LoRA适配器
print("正在添加LoRA适配器...")
try:
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,  # LoRA注意力维度
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                       "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing=False,  # 禁用梯度检查点
        random_state=3407,
        use_rslora=False,  # 不使用rank-stabilized LoRA
        loftq_config=None,  # 不使用LoftQ
    )
    print("LoRA适配器添加完成")
except Exception as e:
    print(f"添加LoRA适配器时出错: {e}")
    raise

# 创建数据整理器
print("正在创建数据整理器...")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 预处理数据集
print("正在预处理数据集...")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

try:
    tokenized_train = train_dataset.map(
        preprocess_function,
        batched=True,
        num_proc=1,
        remove_columns=["text", "prompt", "completion"],
        desc="Running tokenizer on dataset",
    )
    print("数据集预处理完成")
except Exception as e:
    print(f"预处理数据集时出错: {e}")
    raise

# 设置训练参数
print("正在设置训练参数...")
training_args = TrainingArguments(
    output_dir="./medical_model_sft_test",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # 减小批量大小
    gradient_accumulation_steps=16,  # 增加梯度累积步数
    gradient_checkpointing=False,  # 禁用梯度检查点
    optim="adamw_8bit",
    logging_steps=10,
    save_steps=100,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,  # 禁用混合精度训练
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="constant",
    report_to="wandb",
    save_safetensors=True,
    push_to_hub=False,
    # 添加更多参数以避免Triton问题
    no_cuda=False,
    dataloader_drop_last=False,
    dataloader_num_workers=0,  # 禁用数据加载器多进程
    torch_compile=False,  # 禁用torch编译
    use_cpu=False,
    ddp_find_unused_parameters=False,
    tf32=False,
)

# 初始化Trainer
print("正在初始化Trainer...")
try:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    print("Trainer初始化完成")
except Exception as e:
    print(f"初始化Trainer时出错: {e}")
    raise

# 开始训练
print("开始训练...")
start_time = time.time()
try:
    # 直接使用Trainer进行训练
    print("使用Trainer进行训练...")
    trainer.train()
    print(f"训练完成，总时间: {(time.time() - start_time) / 60:.2f}分钟")
except Exception as e:
    print(f"训练过程中出错: {e}")
    print("尝试使用备用训练方法...")
    try:
        # 尝试使用Trainer但关闭所有可能的优化
        print("使用备用Trainer设置...")

        # 重新创建训练参数，禁用所有可能的优化
        backup_training_args = TrainingArguments(
            output_dir="./medical_model_sft_test_backup",
            num_train_epochs=1,  # 减少epoch数量
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,  # 减少梯度累积步数
            gradient_checkpointing=False,
            optim="adamw_torch",  # 使用原生的AdamW优化器
            logging_steps=5,
            save_steps=50,
            learning_rate=2e-4,
            weight_decay=0.001,
            fp16=False,
            bf16=False,
            max_grad_norm=0.3,
            max_steps=20,  # 限制最大步数
            warmup_ratio=0.03,
            group_by_length=False,
            lr_scheduler_type="constant",
            report_to="wandb",
            save_safetensors=True,
            push_to_hub=False,
            no_cuda=False,
            dataloader_drop_last=False,
            dataloader_num_workers=0,
            torch_compile=False,
            use_cpu=False,
            ddp_find_unused_parameters=False,
            tf32=False,
            # 移除不支持的参数
            # sharded_ddp="simple",  # 这个参数可能不被支持
            local_rank=-1,
        )

        # 重新创建Trainer
        backup_trainer = Trainer(
            model=model,
            args=backup_training_args,
            train_dataset=tokenized_train,
            tokenizer=tokenizer,
            data_collator=data_collator,
        )

        # 尝试训练
        backup_trainer.train()
        print(f"备用训练方法成功，总时间: {(time.time() - start_time) / 60:.2f}分钟")
    except Exception as e2:
        print(f"备用训练方法也失败: {e2}")
        print("尝试最简单的训练方法...")
        try:
            # 使用最简单的方法，只训练几个批次
            print("使用最简单的训练方法...")

            # 获取少量数据
            small_dataset = tokenized_train.select(range(min(10, len(tokenized_train))))

            # 准备优化器
            from transformers import AdamW
            optimizer = AdamW(model.parameters(), lr=1e-5)

            # 手动训练循环
            model.train()
            for epoch in range(1):
                print(f"Epoch {epoch+1}")
                for i, batch in enumerate(small_dataset):
                    if i >= 5:  # 只训练5个批次
                        break

                    try:
                        # 准备输入
                        input_ids = torch.tensor([batch['input_ids']]).to(model.device)
                        attention_mask = torch.tensor([batch['attention_mask']]).to(model.device)

                        # 尝试不同的前向传播方式
                        try:
                            # 前向传播
                            with torch.no_grad():  # 先尝试不计算梯度，只测试前向传播
                                test_outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                            print(f"前向传播测试成功，尝试带梯度的前向传播")

                            # 正式前向传播
                            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
                            loss = outputs.loss
                        except Exception as forward_error:
                            print(f"标准前向传播失败: {forward_error}")
                            print("尝试使用替代方法...")

                            # 尝试使用替代方法
                            outputs = model.forward(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
                            loss = outputs.loss

                        # 反向传播
                        loss.backward()

                        # 更新参数
                        optimizer.step()
                        optimizer.zero_grad()

                        print(f"Batch {i+1}, Loss: {loss.item()}")
                    except Exception as batch_error:
                        print(f"处理批次 {i} 时出错: {batch_error}")
                        continue

            print(f"最简单训练方法成功，总时间: {(time.time() - start_time) / 60:.2f}分钟")
        except Exception as e3:
            print(f"所有训练方法都失败: {e3}")
            # 尝试保存当前模型状态，即使训练失败
            try:
                print("尝试保存当前模型状态...")
                model.save_pretrained("./medical_model_sft_partial")
                tokenizer.save_pretrained("./medical_model_sft_partial")
                print("部分训练的模型已保存")
            except Exception as save_error:
                print(f"保存部分训练的模型失败: {save_error}")
            raise

# 保存模型
print("正在保存模型...")
try:
    model.save_pretrained("./medical_model_sft_test_final")
    tokenizer.save_pretrained("./medical_model_sft_test_final")
    print("模型保存完成")
except Exception as e:
    print(f"保存模型时出错: {e}")
    raise

print("SFT训练全部完成！")


已导入triton，版本: 3.1.0
PyTorch版本: 2.5.1+cu124
CUDA是否可用: True
CUDA版本: 12.4
GPU型号: NVIDIA A100-SXM4-40GB
可用GPU内存: 42.47 GB
正在加载数据集...
数据集大小: 25371条

===== 数据集结构检查 =====
数据集特征: {'Question': Value(dtype='string', id=None), 'Complex_CoT': Value(dtype='string', id=None), 'Response': Value(dtype='string', id=None)}
数据集第一个样本: {'Question': 'A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?', 'Complex_CoT': "Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't e

Running tokenizer on dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

数据集预处理完成
正在设置训练参数...
正在初始化Trainer...
Trainer初始化完成
开始训练...
使用Trainer进行训练...


<ipython-input-39-7ade636f0875>:321: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 3 | Total steps = 93
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/3,794,014,208 (1.11% trained)


Step,Training Loss
10,1.219400
20,1.092700
30,1.093300
40,0.991100
50,0.915700
60,0.938400
70,0.837700
80,0.743500
90,0.724900


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in unsloth/mistral-7b-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


训练完成，总时间: 7.68分钟
正在保存模型...


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in unsloth/mistral-7b-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


模型保存完成
SFT训练全部完成！


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
SFT完整数据集训练脚本
使用FreedomIntelligence/medical-o1-reasoning-SFT数据集进行医学模型微调
"""

import os
import gc
import sys
import torch
import random
import inspect
import numpy as np
import transformers
from tqdm import tqdm
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from unsloth import FastLanguageModel
import wandb
from tqdm.auto import tqdm
import time

# 设置随机种子
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# 修复 inspect.findsource 函数，避免递归深度超出限制错误
original_findsource = inspect.findsource

def patched_findsource(object):
    try:
        return original_findsource(object)
    except (OSError, RecursionError):
        # 如果无法获取源代码或递归错误，返回空列表和0
        return [], 0

# 应用补丁
inspect.findsource = patched_findsource

# 设置环境变量以优化训练
os.environ["UNSLOTH_DISABLE_TRITON"] = "1"  # 禁用Triton以避免潜在问题
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 禁用tokenizer并行
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"  # 禁用警告
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # 强制CUDA操作同步执行
os.environ["UNSLOTH_DISABLE_GRADIENT_CHECKPOINTING"] = "1"  # 禁用梯度检查点
os.environ["TORCH_USE_CUDA_DSA"] = "0"  # 禁用CUDA动态共享内存分配
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"  # 限制CUDA内存分配大小
os.environ["PYTORCH_NO_CUDA_MEMORY_CACHING"] = "1"  # 禁用CUDA内存缓存

# 设置PyTorch内存管理
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
if hasattr(torch.backends, "cuda"):
    if hasattr(torch.backends.cuda, "matmul"):
        torch.backends.cuda.matmul.allow_tf32 = False
    if hasattr(torch.backends.cuda, "allow_tf32"):
        torch.backends.cuda.allow_tf32 = False

# 解析命令行参数
def parse_args():
    parser = argparse.ArgumentParser(description="医学模型SFT训练脚本")
    parser.add_argument("--epochs", type=int, default=5, help="训练轮数")
    parser.add_argument("--batch_size", type=int, default=1, help="每个设备的批量大小")
    parser.add_argument("--grad_accum", type=int, default=16, help="梯度累积步数")
    parser.add_argument("-f", "--file", help="Jupyter/Colab内核文件路径，可忽略", default=None)
    # 忽略未知参数
    args, unknown = parser.parse_known_args()

    # 打印参数
    print("\n===== 训练参数 =====")
    for arg in vars(args):
        print(f"{arg}: {getattr(args, arg)}")
    print("===== 训练参数结束 =====\n")

    return args

args = parse_args()

print(f"训练参数: epochs={args.epochs}, batch_size={args.batch_size}, grad_accum={args.grad_accum}")
if unknown:
    print(f"忽略未知参数: {unknown}")

# 内存管理函数
def clean_memory():
    """清理GPU和CPU内存"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def print_memory_stats():
    """打印内存使用情况"""
    print("\n===== 内存使用情况 =====")

    # 打印系统内存使用情况
    try:
        import psutil
        process = psutil.Process(os.getpid())
        memory_info = process.memory_info()
        print(f"系统内存使用: {memory_info.rss / (1024 ** 3):.2f} GB")
    except:
        print("无法获取系统内存使用情况")

    # 打印GPU内存使用情况
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i} 内存分配: {torch.cuda.memory_allocated(i) / (1024 ** 3):.2f} GB")
            print(f"GPU {i} 内存缓存: {torch.cuda.memory_reserved(i) / (1024 ** 3):.2f} GB")
            print(f"GPU {i} 最大内存分配: {torch.cuda.max_memory_allocated(i) / (1024 ** 3):.2f} GB")
    else:
        print("CUDA不可用")

    print("===== 内存使用情况结束 =====\n")

# 检查CUDA是否可用
print("\n===== 环境检查 =====")
print(f"CUDA是否可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA设备数量: {torch.cuda.device_count()}")
    print(f"当前CUDA设备: {torch.cuda.current_device()}")
    print(f"CUDA设备名称: {torch.cuda.get_device_name(0)}")
print("===== 环境检查结束 =====\n")

# 清理内存
clean_memory()
print_memory_stats()

# 设置wandb是否启用
wandb_enabled = True

# 初始化wandb
wandb.init(project="medical-model-grpo", name="medical-sft-full-dataset")

# 加载数据集
print("正在加载数据集...")
try:
    # 加载数据集，指定配置为'zh'（中文）
    full_dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh")["train"]
    print(f"数据集大小: {len(full_dataset)}条")

    # 显示数据集的列
    print(f"数据集列: {full_dataset.column_names}")

    # 显示第一个样本
    print("\n===== 第一个样本 =====")
    for key, value in full_dataset[0].items():
        if isinstance(value, str):
            print(f"{key}: {value[:100]}...")
        else:
            print(f"{key}: {value}")
    print("===== 第一个样本结束 =====\n")
except Exception as e:
    print(f"加载数据集时出错: {e}")
    raise

# 定义系统提示
system_prompt = """你是一个专业的医学AI助手。请根据用户的医学问题，先提供详细的推理过程，然后给出明确的答案。
你的回答必须遵循以下格式：
<reasoning>详细的医学推理过程</reasoning>
<answer>明确的医学答案</answer>
"""

# 处理训练数据
print("开始处理训练数据...")
def process_data(examples):
    """处理数据集中的每个样本"""
    prompts = []
    completions = []

    for i in range(len(examples["Question"])):
        try:
            question = examples["Question"][i]
            complex_cot = examples["Complex_CoT"][i]
            response = examples["Response"][i]

            # 构建提示
            prompt = f"{system_prompt}\n\n用户问题：{question}"

            # 构建完成
            completion = f"<reasoning>{complex_cot}</reasoning>\n<answer>{response}</answer>"

            prompts.append(prompt)
            completions.append(completion)
        except Exception as e:
            print(f"处理样本 {i} 时出错: {e}")
            # 添加空值以保持索引一致
            prompts.append("")
            completions.append("")

    return {"prompt": prompts, "completion": completions}

# 分批处理数据以避免内存问题
batch_size = 100  # 使用较小的批处理大小
num_samples = len(full_dataset)
processed_dataset = {"prompt": [], "completion": []}

for i in range(0, num_samples, batch_size):
    end_idx = min(i + batch_size, num_samples)
    print(f"预处理数据 {i} 到 {end_idx}...")

    # 获取当前批次
    batch_dataset = full_dataset.select(range(i, end_idx))

    # 处理当前批次
    batch_processed = batch_dataset.map(
        process_data,
        batched=True,
        batch_size=batch_size,
        remove_columns=full_dataset.column_names
    )

    # 合并结果
    processed_dataset["prompt"].extend(batch_processed["prompt"])
    processed_dataset["completion"].extend(batch_processed["completion"])

    # 每5000个样本清理一次内存
    if (i + batch_size) % 5000 == 0 or end_idx == num_samples:
        print(f"已处理 {end_idx} 条数据，清理内存...")
        gc.collect()
        print_memory_stats()

# 创建最终数据集
train_dataset = Dataset.from_dict(processed_dataset)
print(f"准备完成，共{len(train_dataset)}条数据")
print(f"创建训练数据集成功，大小: {len(train_dataset)}条")
print_memory_stats()

# 加载模型
print("正在加载模型...")
try:
    # 设置torch数据类型
    torch_dtype = torch.bfloat16

    # 使用FastLanguageModel加载模型，禁用一些可能导致问题的功能
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/mistral-7b-bnb-4bit",
        max_seq_length=2048,
        dtype=torch_dtype,
        load_in_4bit=True,
        # 禁用可能导致递归问题的功能
        use_gradient_checkpointing=False,
        use_cache=True,
    )

    # 添加LoRA适配器
    print("正在添加LoRA适配器...")
    model = FastLanguageModel.get_peft_model(
        model,
        r=32,  # LoRA注意力维度
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                         "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing=False,  # 禁用梯度检查点以避免递归问题
        random_state=42,
        use_rslora=False,  # 是否使用rank-stabilized LoRA
        loftq_config=None,  # 是否使用LoftQ
    )
    print("LoRA适配器添加完成")

    # 显示第一个处理成功的样本
    try:
        first_sample = process_data({"Question": [full_dataset[0]["Question"]], "Complex_CoT": [full_dataset[0]["Complex_CoT"]], "Response": [full_dataset[0]["Response"]]})
        print("\n===== 第一个处理成功的样本 =====")
        print(f"提示: {first_sample['prompt'][0][:100]}...")
        print(f"回复: {first_sample['completion'][0][:100]}...")
        print("===== 第一个处理成功的样本结束 =====\n")
    except Exception as e:
        print(f"处理第一个样本时出错: {e}")

    # 预处理数据集
    print("正在预处理数据集...")
    def preprocess_function(examples):
        batch_size = len(examples["prompt"])
        model_inputs = {
            "input_ids": [],
            "attention_mask": [],
            "labels": []
        }

        # 处理每个样本
        for i in range(batch_size):
            try:
                prompt = examples["prompt"][i]
                completion = examples["completion"][i]

                # 对提示进行编码
                prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)

                # 对完成进行编码
                completion_ids = tokenizer.encode(completion, add_special_tokens=False)

                # 确保总长度不超过模型最大长度 (2048)
                max_length = 2048 - 1  # 减1是为了留出结束标记的空间

                # 如果总长度超过限制，优先保留完成部分，截断提示部分
                if len(prompt_ids) + len(completion_ids) > max_length:
                    # 确保至少保留完成部分和一些提示上下文
                    completion_length = min(len(completion_ids), max_length // 2)
                    prompt_length = max_length - completion_length

                    # 截断
                    prompt_ids = prompt_ids[-prompt_length:] if len(prompt_ids) > prompt_length else prompt_ids
                    completion_ids = completion_ids[:completion_length] if len(completion_ids) > completion_length else completion_ids

                    print(f"警告：样本 {i} 被截断，原始长度：{len(prompt_ids) + len(completion_ids) + 1}，截断后长度：{len(prompt_ids) + len(completion_ids) + 1}")

                # 组合输入ID
                input_ids = prompt_ids + completion_ids + [tokenizer.eos_token_id]

                # 创建标签，提示部分用-100掩码，只预测完成部分
                labels = [-100] * len(prompt_ids) + completion_ids + [tokenizer.eos_token_id]

                # 创建注意力掩码
                attention_mask = [1] * len(input_ids)

                # 添加到批次
                model_inputs["input_ids"].append(input_ids)
                model_inputs["attention_mask"].append(attention_mask)
                model_inputs["labels"].append(labels)
            except Exception as e:
                print(f"处理样本 {i} 时出错: {e}")
                # 添加一个空样本以保持索引一致
                model_inputs["input_ids"].append([tokenizer.pad_token_id])
                model_inputs["attention_mask"].append([0])
                model_inputs["labels"].append([-100])

        return model_inputs

    # 分批预处理数据集
    print("开始分批预处理数据集...")
    batch_size = 100  # 每批处理的样本数
    num_batches = (len(train_dataset) + batch_size - 1) // batch_size

    tokenized_datasets = []
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(train_dataset))
        print(f"预处理数据 {start_idx} 到 {end_idx}...")

        # 获取当前批次的数据
        batch_dataset = train_dataset.select(range(start_idx, end_idx))

        # 对当前批次进行预处理
        tokenized_batch = batch_dataset.map(
            preprocess_function,
            batched=True,
            desc=f"Running tokenizer on batch {start_idx}-{end_idx}"
        )

        tokenized_datasets.append(tokenized_batch)

        # 每5个批次清理一次内存
        if (i + 1) % 5 == 0 or i == num_batches - 1:
            print(f"已预处理 {end_idx}/{len(train_dataset)} 条数据，清理内存...")
            clean_memory()

    # 合并所有预处理后的批次
    tokenized_dataset = concatenate_datasets(tokenized_datasets) if len(tokenized_datasets) > 1 else tokenized_datasets[0]
    print(f"预处理完成，共 {len(tokenized_dataset)} 条数据")

    # 清理内存
    clean_memory()
    print_memory_stats()

except Exception as e:
    print(f"加载模型时出错: {e}")
    raise

# 内存清理回调
class MemoryCleanupCallback(transformers.TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0:  # 每100步清理一次内存
            clean_memory()

    def on_epoch_end(self, args, state, control, **kwargs):
        clean_memory()
        print(f"Epoch {state.epoch} 完成，清理内存...")
        print_memory_stats()

# 创建自定义数据整理器
class CustomDataCollator:
    def __init__(self, tokenizer, torch_dtype=torch.bfloat16):
        self.tokenizer = tokenizer
        self.pad_token_id = tokenizer.pad_token_id
        self.torch_dtype = torch_dtype

    def __call__(self, features):
        # 确保features是列表
        if not isinstance(features, list):
            features = [features]

        # 如果列表为空，返回空字典
        if len(features) == 0:
            return {}

        # 获取批次中所有样本的长度
        batch_size = len(features)

        # 提取input_ids, attention_mask和labels
        input_ids = [feature["input_ids"] for feature in features]
        attention_mask = [feature["attention_mask"] for feature in features]
        labels = [feature["labels"] for feature in features]

        # 找出最大长度
        max_length = max(len(ids) for ids in input_ids)

        # 填充input_ids
        padded_input_ids = []
        padded_attention_mask = []
        padded_labels = []

        for i in range(batch_size):
            # 当前样本的长度
            curr_len = len(input_ids[i])

            # 需要填充的长度
            padding_len = max_length - curr_len

            # 填充input_ids
            padded_input_ids.append(input_ids[i] + [self.pad_token_id] * padding_len)

            # 填充attention_mask
            padded_attention_mask.append(attention_mask[i] + [0] * padding_len)

            # 填充labels
            padded_labels.append(labels[i] + [-100] * padding_len)

        # 转换为张量
        input_ids_tensor = torch.tensor(padded_input_ids, dtype=torch.long)
        attention_mask_tensor = torch.tensor(padded_attention_mask, dtype=torch.long)
        labels_tensor = torch.tensor(padded_labels, dtype=torch.long)

        # 返回批次
        return {
            "input_ids": input_ids_tensor,
            "attention_mask": attention_mask_tensor,
            "labels": labels_tensor
        }

# 使用Unsloth的内置训练方式
print("开始使用Unsloth的内置训练方式...")
try:
    # 创建数据整理器
    data_collator = CustomDataCollator(tokenizer, torch_dtype=torch_dtype)

    # 设置训练参数
    training_args = TrainingArguments(
        output_dir="./models/sft_model",
        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.batch_size,
        gradient_accumulation_steps=args.grad_accum,
        learning_rate=2e-5,
        weight_decay=0.01,
        warmup_steps=100,
        logging_steps=10,
        save_steps=1000,
        save_total_limit=3,
        fp16=False,
        bf16=True,
        remove_unused_columns=False,
        report_to="wandb" if wandb_enabled else "none",
        optim="adamw_torch",
        lr_scheduler_type="cosine",
        seed=42,
    )

    # 创建Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator,
        callbacks=[MemoryCleanupCallback()],
    )

    # 开始训练
    print("开始训练...")
    trainer.train()

    # 保存最终模型
    print("训练完成，保存最终模型...")
    trainer.save_model("./medical_model_sft_full_final")
    tokenizer.save_pretrained("./medical_model_sft_full_final")
    print("模型保存完成")

except Exception as e:
    print(f"训练过程中出错: {e}")
    # 尝试保存模型
    try:
        output_dir = "./models/sft_model_error"
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        print(f"模型已保存到 {output_dir}")
    except Exception as save_error:
        print(f"保存模型时出错: {save_error}")

# 清理内存
clean_memory()
print_memory_stats()

print("SFT完整数据集训练全部完成！")
print(f"总训练时间: {(time.time() - start_time) / 60:.2f}分钟")
print(f"总训练时间: {(time.time() - start_time) / 3600:.2f}小时")

# 关闭wandb
if wandb_enabled:
    wandb.finish()



===== 训练参数 =====
epochs: 5
batch_size: 1
grad_accum: 16
file: /root/.local/share/jupyter/runtime/kernel-ab648c7a-1719-4d2a-8839-8eb1d5dc1764.json
===== 训练参数结束 =====

训练参数: epochs=5, batch_size=1, grad_accum=16

===== 环境检查 =====
CUDA是否可用: True
CUDA设备数量: 1
当前CUDA设备: 0
CUDA设备名称: NVIDIA A100-SXM4-40GB
===== 环境检查结束 =====


===== 内存使用情况 =====
系统内存使用: 8.31 GB
GPU 0 内存分配: 18.96 GB
GPU 0 内存缓存: 19.24 GB
GPU 0 最大内存分配: 25.03 GB
===== 内存使用情况结束 =====

正在加载数据集...
数据集大小: 24772条
数据集列: ['Question', 'Complex_CoT', 'Response']

===== 第一个样本 =====
Question: 根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？...
Complex_CoT: 这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节...
Response: 从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的...
===== 第一个样本结束 =====

开始处理训练数据...
预处理数据 0 到 100...
预处理数据 100 到 200...
预处理数据 200 到 300...
预处理数据 300 到 400...
预处理数据 400 到 500...
预处理数据 500 到 600...

Running tokenizer on batch 0-100:   0%|          | 0/100 [00:00<?, ? examples/s]

预处理数据 100 到 200...


Running tokenizer on batch 100-200:   0%|          | 0/100 [00:00<?, ? examples/s]

预处理数据 200 到 300...


...

Step,Training Loss
10,1.371600
20,1.366400
30,1.349100
40,1.314500
50,1.307000
60,1.251500
70,1.236800
80,1.240600
90,1.210200
100,1.175400


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in unsloth/mistral-7b-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


Epoch 1.0 完成，清理内存...

===== 内存使用情况 =====
系统内存使用: 8.37 GB
GPU 0 内存分配: 20.40 GB
GPU 0 内存缓存: 20.49 GB
GPU 0 最大内存分配: 28.41 GB
===== 内存使用情况结束 =====



/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in unsloth/mistral-7b-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in unsloth/mistral-7b-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


Epoch 2.0 完成，清理内存...

===== 内存使用情况 =====
系统内存使用: 8.37 GB
GPU 0 内存分配: 20.40 GB
GPU 0 内存缓存: 20.49 GB
GPU 0 最大内存分配: 28.41 GB
===== 内存使用情况结束 =====



And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 83,886,080


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
</reasoning>Assume Ahmed's grade in the remaining 9 assignments is x. Emily's total grade in the first 9 assignments is 92. We can represent this as 92 + the grade on the final assignment. Since the final assignment is worth the same as the other assignments, we can say the grade on the final assignment is 0.9x, since it is 90% of the total of the other assignments. Now, we can write the equation for the total grades as follows: 

92 + 0.9x = 1x + 90 

Subtracting 92 from both sides gives us 0.9x = x - 2. 
Subtracting 0.9x from both sides gives us 0 = 0.1x - 2. Subse

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.000000,0.000000,196.500000,0.000000
2,0.000000,0.040667,0.099613,183.500000,0.000000
3,0.000000,-0.019833,0.035000,137.500000,0.000005
4,0.000000,0.409833,1.075630,188.000000,0.000007
5,0.000000,0.020333,0.049806,198.000000,0.000004
6,0.000000,0.000833,0.114188,140.666672,0.000006
7,0.000000,-0.028000,0.068586,140.833344,0.000009
8,0.000000,0.400833,1.029079,139.000000,0.000007
9,0.000000,0.000000,0.000000,184.333344,0.000009
10,0.000000,0.000000,0.000000,144.000000,0.000011


-------------------- Question:
The gauge on a water tank shows that the tank is 1/3 full of water. To fill the tank, 16 gallons of water are added. How many gallons of water does the tank hold when full? 
Answer:
24 
Response:
Let's break this problem down. 

If the tank is 1/3 full and 16 gallons are added to fill it, then the amount of water added is equal to the amount that was already in the tank (1/3 of the total capacity) plus the amount needed to fill it (2/3 of the total capacity). 

Let x be the capacity of the tank in gallons. 

The equation becomes x - (1/3)x + 16 = x.

To simplify the equation, we can combine the x terms: (2/3)x + 16 = x.

Next, subtract (2/3)x from both sides of the equation: (1/3)x = 16.

Now, multiply both sides of the equation by 3 to get rid of the fraction: x = 16 * 3

x = 48

The tank holds 48 gallons of water when full. 
Extracted:
Let's break this problem down. 

If the tank is 1/3 full and 16 gallons are added to fill it, then the amount of water 

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.000000,0.000000,196.500000,0.000000
2,0.000000,0.040667,0.099613,183.500000,0.000000
3,0.000000,-0.019833,0.035000,137.500000,0.000005
4,0.000000,0.409833,1.075630,188.000000,0.000007
5,0.000000,0.020333,0.049806,198.000000,0.000004
6,0.000000,0.000833,0.114188,140.666672,0.000006
7,0.000000,-0.028000,0.068586,140.833344,0.000009
8,0.000000,0.400833,1.029079,139.000000,0.000007
9,0.000000,0.000000,0.000000,184.333344,0.000009
10,0.000000,0.000000,0.000000,144.000000,0.000011


-------------------- Question:
Felix is chopping down trees in his backyard. For every 13 trees he chops down he needs to get his axe resharpened. It cost him $5 to sharpen his axe. If he spends $35 on axe sharpening, at least how many trees has he chopped down? 
Answer:
91 
Response:
Let's start by finding the number of times Felix got his axe sharpened. Since sharpening costs $5 each, and he spent $35 in total, he got his axe sharpened $35 / $5 = 7 times.

If for every 13 trees he chops down, he needs to get his axe sharpened, then after 7 sharpenings, he must have chopped down 7 * 13 = 91 trees.

<answer>91</answer> 
Extracted:
91
-------------------- Question:
Laura wants to bake a cake for her mother. She checks the recipe and the pantry and sees that she needs to buy 2 cups of flour, 2 cups of sugar, a cup of butter, and two eggs. The flour costs $4. The sugar costs $2. The eggs cost $.5, and the butter costs $2.5. When she is done baking it, she cuts the cake into 6 slices. Her 

TrainOutput(global_step=250, training_loss=0.001432442097942186, metrics={'train_runtime': 11046.913, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.023, 'total_flos': 0.0, 'train_loss': 0.001432442097942186})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.78s/it, est. speed input: 1.64 toks/s, output: 19.94 toks/s]


'Calculating pi to a large number of decimal places is a complex task that requires a computational approach, rather than a simple mathematical formula. Here\'s a way to calculate pi using the Monte Carlo method, which is an approximation method that uses random numbers to estimate the value of pi:\n\n**The Monte Carlo Method**\n\nThe Monte Carlo method is based on the idea of simulating the probability of a random walk across a square and circle. Here\'s the basic idea:\n\n1. Draw a square and a circle on a piece of paper.\n2. Generate random points within the square.\n3. Count the proportion of points that fall within the circle.\n4. The ratio of points within the circle to the total number of points is approximately equal to the ratio of the area of the circle to the area of the square, which is pi.\n\n**Mathematical Formulation**\n\nLet\'s denote the following variables:\n\n*   `N`: the number of random points generated\n*   `n`: the number of points within the circle\n*   `pi_appr

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.29s/it, est. speed input: 2.62 toks/s, output: 19.41 toks/s]


"<reasoning>\nPi (π) is an irrational number that represents the ratio of a circle's circumference to its diameter. It is approximately equal to 3.14159, but its decimal representation goes on indefinitely without repeating.\n\nTo calculate pi, we can use various mathematical formulas and methods, such as the Leibniz formula, the Gregory-Leibniz series, or the Monte Carlo method. However, these methods are not practical for obtaining a high degree of accuracy.\n\nA more practical approach is to use the Bailey-Borwein-Plouffe (BBP) formula, which is a spigot algorithm that allows us to calculate any digit of pi without having to compute the preceding digits.\n\nAnother method is to use the Chudnovsky algorithm, which is a fast and efficient method for calculating pi to a high degree of accuracy.\n\nFor simplicity, we can use the first few terms of the BBP formula to estimate pi:\nπ = 3 + 1/(4/3 - 1/(4/3 - 1/(4/3 - ...))\n\nLet's use this simplified formula to estimate pi:\n\nπ ≈ 3 + 1/(

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
